## Notebook goal: scrape a little more from each of the books on the booktok shelf on Goodreads (use for joins on the Kindle dataset)

Scraping: Genre (top genre only), Edition (not sure if I'll use it, but let's grab for now), Number of pages, Publisher, Goodreads Choice nominee flag 

In [3]:
import requests
from requests import get 
from bs4 import BeautifulSoup as BS
import re
import pandas as pd
import numpy as np
import time
from time import sleep
from random import randint
from random import uniform 
import matplotlib.pyplot as plt
%matplotlib inline
import lxml 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service  # Import Service for Chrome
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

driver.get("https://www.goodreads.com/user/sign_in")

button = driver.find_element(By.LINK_TEXT, "Sign in with email")
button.click()

username = driver.find_element(By.NAME, "email")
username.send_keys("jennifer.toppins@gmail.com")

In [4]:
import json
# with open('credentials.json', 'w') as f:
#     json.dump(credentials, f) #need to put my credentials in a file to use later...

with open('credentials.json') as f:
    loaded_credentials = json.load(f)

time.sleep(10) #adding some sleep in here because I got the MaxRetryError as I was practicing this 

In [5]:
email = loaded_credentials['email']
password = loaded_credentials['password']

In [6]:
password_field = driver.find_element(By.NAME, "password")
password_field.send_keys(password)

In [7]:
signin = driver.find_element(By.ID, "signInSubmit")
signin.click()

Read in the df I already scraped

In [9]:
booktok_books = pd.read_csv(r"C:\Users\jenni\OneDrive\Desktop\DA12\Projects\2024-DA12-Goodreads-capstone\data\booktok_books_df_final2.csv")

In [10]:
booktok_books.head()

,Title,Author,Num_shelved_booktok,Goodreads URL,Image URL,avg rating,num_ratings,year_published
0,The Seven Husbands of Evelyn Hugo (Hardcover),Taylor Jenkins Reid,1023,https://www.goodreads.com/book/show/32620332-t...,https://i.gr-assets.com/images/S/compressed.ph...,4.41,"3,340,727",2017.0
1,It Ends with Us (It Ends with Us #1),Colleen Hoover,875,https://www.goodreads.com/book/show/27362503-i...,https://i.gr-assets.com/images/S/compressed.ph...,4.14,"3,863,992",2016.0
2,The Song of Achilles (Paperback),Madeline Miller,806,https://www.goodreads.com/book/show/13623848-t...,https://i.gr-assets.com/images/S/compressed.ph...,4.32,"1,628,438",2011.0
3,"The Cruel Prince (The Folk of the Air, #1)",Holly Black,783,https://www.goodreads.com/book/show/26032825-t...,https://i.gr-assets.com/images/S/compressed.ph...,4.03,"1,378,006",2018.0
4,A Court of Thorns and Roses (A Court of Thorns...,Sarah J. Maas,761,https://www.goodreads.com/book/show/50659467-a...,https://i.gr-assets.com/images/S/compressed.ph...,4.18,"3,068,237",2015.0


In [11]:
#grab the urls for scraping
urls = booktok_books['Goodreads URL'].tolist()
print(urls)

['https://www.goodreads.com/book/show/32620332-the-seven-husbands-of-evelyn-hugo', 'https://www.goodreads.com/book/show/27362503-it-ends-with-us', 'https://www.goodreads.com/book/show/13623848-the-song-of-achilles', 'https://www.goodreads.com/book/show/26032825-the-cruel-prince', 'https://www.goodreads.com/book/show/50659467-a-court-of-thorns-and-roses', 'https://www.goodreads.com/book/show/56732449-the-love-hypothesis', 'https://www.goodreads.com/book/show/50623864-the-invisible-life-of-addie-larue', 'https://www.goodreads.com/book/show/23437156-six-of-crows', 'https://www.goodreads.com/book/show/52831200-from-blood-and-ash', 'https://www.goodreads.com/book/show/41150487-red-white-royal-blue', 'https://www.goodreads.com/book/show/40916679-a-good-girl-s-guide-to-murder', 'https://www.goodreads.com/book/show/33385229-they-both-die-at-the-end', 'https://www.goodreads.com/book/show/52439531-the-inheritance-games', 'https://www.goodreads.com/book/show/61431922-fourth-wing', 'https://www.go

In [12]:
len(urls) #check to make sure they're all there 

1250

In [13]:
#scrape book info from each url
meta = []
top_genre = []

for i in range(len(urls)):
    sleep(uniform(1,6)) #random sleep to keep from getting kicked out again
    url = urls[i]
    driver.get(url)  
    soup = BS(driver.page_source, 'lxml')
    try:
        details = soup.find_all('p', {'data-testid': 'pagesFormat'})[0]
        meta.append(details.text.strip())
    except IndexError:
        print(f'No details found for {url}')
        meta.append(np.nan)
    genre = soup.find_all('span', {'class': 'BookPageMetadataSection__genreButton'})
    if len(genre) < 1:
        continue 
    genre2 = genre[0]
    genre3 = genre2.find('span', 'Button__labelItem')
    genre4 = genre3.text.strip()
    
    #meta.append(details.text.strip())
    top_genre.append(genre4)

No details found for https://www.goodreads.com/book/show/25574782-pretty-girls
No details found for https://www.goodreads.com/book/show/52694527-the-shadow-of-the-gods
No details found for https://www.goodreads.com/book/show/54256265-chasing-love
No details found for https://www.goodreads.com/book/show/124943221-king-of-greed
No details found for https://www.goodreads.com/book/show/53418394-for-the-wolf
No details found for https://www.goodreads.com/book/show/59752535-don-t-let-me-fall
No details found for https://www.goodreads.com/book/show/49399658-counting-down-with-you
No details found for https://www.goodreads.com/book/show/60641657-the-four-leaf
No details found for https://www.goodreads.com/book/show/203156729-the-spymaster
No details found for https://www.goodreads.com/book/show/62881797-iced-out
No details found for https://www.goodreads.com/book/show/195138739-house-of-bane-and-blood
No details found for https://www.goodreads.com/book/show/53223710-heaven
No details found for

In [34]:
print(meta)

['389 pages, Hardcover', '386 pages, Kindle Edition', '408 pages, Paperback', '370 pages, Hardcover', '419 pages, Paperback', '383 pages, Paperback', '448 pages, Hardcover', '480 pages, Hardcover', '622 pages, ebook', '448 pages, Paperback', '433 pages, Paperback', '389 pages, Hardcover', '386 pages, Hardcover', '517 pages, Hardcover', '336 pages, ebook', '361 pages, Paperback', '364 pages, Paperback', '338 pages, Hardcover', '487 pages, ebook', '330 pages, Kindle Edition', '242 pages, Kindle Edition', '360 pages, Kindle Edition', '435 pages, Kindle Edition', '626 pages, Paperback', '407 pages, Hardcover', '368 pages, Hardcover', '406 pages, Hardcover', '448 pages, Paperback', '358 pages, Hardcover', '377 pages, Hardcover', '432 pages, Paperback', '393 pages, Hardcover', '372 pages, Hardcover', '273 pages, Hardcover', '438 pages, Kindle Edition', '512 pages, Hardcover', '288 pages, Hardcover', '365 pages, Paperback', '388 pages, Hardcover', '572 pages, Kindle Edition', '397 pages, Pape

In [36]:
print(len(meta)) #i'm going to try something with the Nans a bit later...

1250


In [38]:
print(top_genre)

['Fiction', 'Fiction', 'Fantasy', 'Fantasy', 'Fantasy', 'Contemporary', 'Fantasy', 'Fantasy', 'Romance', 'Romance', 'Young Adult', 'Young Adult', 'Mystery', 'Dragons', 'Thriller', 'Romance', 'Fiction', 'Dystopia', 'Contemporary', 'Romance', 'Young Adult', 'Romance', 'Sports Romance', 'Fantasy', 'Fantasy', 'Fiction', 'Young Adult', 'Contemporary', 'Fantasy', 'Romance', 'Contemporary', 'Fantasy', 'Young Adult', 'Fiction', 'Mythology', 'Romance', 'Fiction', 'Romance', 'Young Adult', 'Contemporary', 'Contemporary', 'Romance', 'Historical Fiction', 'New Adult', 'Fantasy', 'Fantasy', 'Dark', 'Romance', 'Fiction', 'Fantasy', 'Mystery', 'Romance', 'Young Adult', 'Fantasy', 'Fiction', 'Fantasy', 'Contemporary', 'Fantasy', 'Thriller', 'Fantasy', 'Fantasy', 'Romance', 'Romance', 'Fantasy', 'Mystery', 'Fantasy', 'Romance', 'Romance', 'Romance', 'Fantasy', 'Fantasy', 'Fantasy', 'Fantasy', 'Romance', 'Fantasy', 'Fiction', 'Romance', 'Fantasy', 'Fantasy', 'Fantasy', 'Romance', 'Fantasy', 'Romance', '

In [40]:
print(len(top_genre)) #as suspected, missing 10 genres...

1240


In [44]:
#scrape book info from each url - scrape again to have a list to compare/add to for the meta (think that it's just a connection issue)
#plus need to fix the genre problem 
meta2 = []
top_genre2 = []

for i in range(len(urls)):
    sleep(uniform(1,6)) #random sleep to keep from getting kicked out again
    new_url = urls[i]
    driver.get(new_url)  
    new_soup = BS(driver.page_source, 'lxml')
    try:
        new_details = soup.find_all('p', {'data-testid': 'pagesFormat'})[0]
        meta2.append(new_details.text.strip())
    except IndexError:
        print(f'No details found for {url}')
        meta2.append(np.nan)
    new_genre = soup.find_all('span', {'class': 'BookPageMetadataSection__genreButton'})
    if len(new_genre) > 0:
        new_genre2 = new_genre[0] #select only the first genre
        new_genre3 = new_genre2.find('span', 'Button__labelItem')
        new_genre4 = new_genre3.text.strip() if genre3 is not None else np.nan
        top_genre2.append(new_genre4)
    else:
        print(f'No genres found for {url}')
        top_genre2.append(np.nan) #throw nans if it's not found

In [46]:
len(meta2)

1250

In [48]:
len(top_genre2)

1250

In [50]:
print(meta2)

['326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Paperback', '326 pages, Pap

In [52]:
top_genre2

['Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Romance',
 'Ro

Well that didn't work either. Need to tweak again.

In [54]:
driver.quit()

In [16]:
#find the indices of the missing books

#urls.index('https://www.goodreads.com/book/show/51335759-good-girl-bad-blood') #107
#urls.index('https://www.goodreads.com/book/show/50548197-a-deadly-education') #284
#urls.index('https://www.goodreads.com/book/show/61435016-fractured-freedom') #317
#urls.index('https://www.goodreads.com/book/show/58756420-i-kissed-shara-wheeler') #499
#urls.index('https://www.goodreads.com/book/show/59752535-don-t-let-me-fall') #547
#urls.index('https://www.goodreads.com/book/show/198966989-things-i-wanted-to-say') #597
#urls.index('https://www.goodreads.com/book/show/53487148-a-thousand-ships') #600
# urls.index('https://www.goodreads.com/book/show/61414050-dirty-wicked-prince') #612
#urls.index('https://www.goodreads.com/book/show/60444543-the-confidence-of-wildflowers') #780
#urls.index('https://www.goodreads.com/book/show/60641657-the-four-leaf') #861
#urls.index('https://www.goodreads.com/book/show/203156729-the-spymaster') #862
#urls.index('https://www.goodreads.com/book/show/62881797-iced-out') #1072
urls.index('https://www.goodreads.com/book/show/57992291-unlawful-temptations') ##1170

1170

In [17]:
# missing_indices = [107, 284, 317, 499, 547, 597, 600, 612, 780, 861, 862, 1072, 1170]
# len(missing_indices)

In [18]:
# number_urls = 1250
# current_meta_length = len(meta)

# for index in sorted(missing_indices, reverse=True):
#     meta.insert(index, np.nan)  # Add NaN to the meta list
#     #top_genre.append(np.nan)  # Add NaN to the top genre list

# Now, the meta and top_genre lists will have NaN appended if they were shorter than num_urls
#print("Meta:", meta)
#print("Top Genre:", top_genre)